# Cost Management APIs
This notebook can show you how to export costs and use them programatically

- [API Reference](https://learn.microsoft.com/en-us/rest/api/cost-management/generate-cost-details-report/create-operation?view=rest-cost-management-2023-11-01&tabs=HTTP#generatecostdetailsreportbysubscriptionandtimeperiod)

> Assume you have a `.env` for your `AZURE_SUBSCRIPTION_ID`

In [1]:
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os
from datetime import datetime,timedelta
import requests
import json
from tabulate import tabulate 
import pandas as pd


load_dotenv()

subId=os.getenv('AZURE_SUBSCRIPTION_ID')

credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default").token
headers= {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}

In [ ]:
#Get the first day of the month in yyyy-MM-dd format
dtStart = datetime.now().replace(day=1).strftime('%Y-%m-%d')
dtCurrent=datetime.now().strftime('%Y-%m-%d')
dtPrevious=(datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
dtMonthPrevious=(datetime.now().replace(day=1) - timedelta(days=1)).strftime('%Y-%m-%d')

print(dtStart)
print(dtPrevious)

body={
    "metric": "ActualCost",
    "timePeriod": {
        "start": dtStart,
        "end": dtPrevious
    }
}

#convert body to utf8 bytes
body_utf8 = bytes(json.dumps(body), 'utf-8')

url = f"https://management.azure.com/subscriptions/{subId}/providers/Microsoft.CostManagement/generateCostDetailsReport?api-version=2023-11-01"
response = requests.post(url, headers=headers,json=body)
print(response.status_code)

location=response.headers['Location']
print("Location:",location)
url=location

In [4]:
# Run until you get a 200
response = requests.get(url, headers=headers)
print(response.status_code)
if 200 != response.status_code:
    print(response.content)

200


In [ ]:
if 200 == response.status_code:
    content=response.json()
    print(json.dumps(content, indent=4, sort_keys=True))
    blobLink=content['manifest']['blobs'][0]['blobLink']
    print(blobLink)

In [6]:
#Download blob
response = requests.get(blobLink)

# Write to a csv
with open('./local/cost.csv', 'wb') as f:
    f.write(response.content)

In [7]:
# load csv into pandas and display
df = pd.read_csv('./local/cost.csv')
# filter where meterSubCategory eq 'Virtual Machines'
# df = df[df['meterSubCategory'] == 'Azure OpenAI']
# Print the columns meterName, meterSubCategory, usageStart, usageEnd, pretaxCost
# print(df[['meterName', 'meterSubCategory', 'ProductName', 'location', 'effectivePrice','quantity','costInBillingCurrency','unitOfMeasure']])
# Aggregate the quantity and costInBillingCurrency by meterName,location

# print without wrapping
pd.set_option('display.expand_frame_repr', False)
# print(df.groupby(['location','meterName'])[['quantity','costInBillingCurrency']].sum())
print(df.groupby(['location','ProductName'])[['quantity','costInBillingCurrency']].sum())
# print(df.head(10))


                                                                       quantity  costInBillingCurrency
location         ProductName                                                                          
AU East          Azure OpenAI - gpt-35-turbo-4k-Input-regional -...    0.114000               0.000171
                 Azure OpenAI - gpt-35-turbo-4k-Output-regional ...    0.561000               0.001122
CA East          Azure OpenAI - gpt-35-turbo-4k-Input-regional -...    0.312000               0.000468
                 Azure OpenAI - gpt-35-turbo-4k-Output-regional ...    1.654000               0.003308
CH North         Azure OpenAI - gpt-35-turbo-4k-Input-regional -...    0.086000               0.000129
...                                                                         ...                    ...
US South Central Azure OpenAI - Az-GPT-3.5-turbo                       0.092000               0.000184
global           Azure DNS - Private                                   0.